# 🪄 Fine-tuning and evaluating GPT-3.5 with human feedback for RAG 

This guide explains how to fine-tune OpenAI's GPT3.5-turbo with your own data and Argilla to improve a RAG (Retrieval Augmented Generation) system. 

It includes the following steps:

- Setting up a RAG pipeline using [LlamaIndex](https://github.com/jerryjliu/llama_index) and [Unstructured](https://github.com/Unstructured-IO/unstructured) to answer questions using a document about Argilla Cloud.
- Generating potential questions with LlamaIndex to build a training and test set.
- Building a dataset for collecting human written responses with Argilla.
- Fine-tuning GPT3.5-turbo with high-quality data.
- Evaluating the fine-tuned model vs the base model with human preference data from Argilla.

The goal of the tutorial is to demonstrate how to incorporate human feedback into your LLM development for two critical stages: 

1. Gathering **high quality data for fine-tuning**, 
2. Gathering **human feedback for evaluation of LLM applications**.


Given the ongoing debate between Retrieval Augmented Generation (RAG) and fine-tuning, we selected a real-world RAG use case to demonstrate how fine-tuning enhances the style, utility, and relevance of responses within a RAG application. The resulting system will be Hybrid RAG system (RAG using fine-tuned models) as [described in this article](https://towardsdatascience.com/rag-vs-finetuning-which-is-the-best-tool-to-boost-your-llm-application-94654b1eaba7). 

The screenshot below displays the evaluation dataset, termed the "human preference dataset." In it, `response-1` is produced by the fine-tuned model, while `response-2` comes from the base GPT-3.5 model. With just minor fine-tuning and without altering the system message, we've directed the LLM's behavior towards generating responses that are more helpful, faithful, friendly, and aligned with our brand.

Fine-tuning effectively mitigates common RAG challenges, like the LLM referring to the context using phrases such as "The context does not provide information about this." This enhancement is notable even when we had incorporated directives in the system message to deter such references, like "2. Avoid phrases such as 'Based on the context, ...' or 'The context information ...'." (see Llama Index default prompt later).


![Human preference](../../../_static/images/llms/fine-tuning-openai-rag-feedback/preference-fine-tuned.png "Fine-tuned model vs base model response")




By the end of the tutorial, you'll be using a fine-tuned model for RAG and have an human evaluation workflow in place to continously evaluate your LLM application (see below for a comparison of the base gpt3.5 vs the fine-tuned gpt3.5 for this application).

![Fine-tuned model vs base model](../../../_static/images/llms/fine-tuning-openai-rag-feedback/fine-tune-vs-base.png "Fine-tuned model vs base model")


Let's get started!

## Setup

For running this tutorial, you need to [install and launch Argilla](https://docs.argilla.io/en/latest/getting_started/quickstart_installation.html), as well as some other packages.

In [ ]:
%pip install argilla openai datasets llama-index unstructured -qqq

In [ ]:
# imports
import os
import json
import random
from tqdm import tqdm
import time
import matplotlib.pyplot as plt

import openai

import argilla as rg

from llama_index import SimpleDirectoryReader, ServiceContext, download_loader
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator
from llama_index import VectorStoreIndex

from datasets import load_dataset

In [ ]:
# connect to argilla quickstart instance using your URL and the default api key (literally admin.apikey)
# in this example we are using a hugging face deployment.
rg.init(
    api_url="https://argilla-fine-tune-chat-gpt.hf.space/",
    api_key="admin.apikey"
)

In [ ]:
# Your openai key is needed for generation and fine-tuning
os.environ['OPENAI_API_KEY'] = 'sk-...'
openai.api_key = os.environ["OPENAI_API_KEY"]

## Generating responses with LlamaIndex and GPT3.5

We generate responses for the generated questions using [this dataset about Argilla Cloud](https://huggingface.co/datasets/argilla/cloud_assistant_questions). We have generated this dataset using a source document and LlamaIndex's question generator (see appendix about how to generate these questions).

If you want to skip this process (it will take several minutes), we have shared the resulting [dataset on Hugging Face](https://huggingface.co/datasets/argilla/rg_customer_assistant).





In [ ]:
# read our source questions
dataset = load_dataset("argilla/cloud_assistant_questions")

In [ ]:
# read and parse document using Unstructured
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)
loader = UnstructuredReader()
# You can download this doc from: https://huggingface.co/datasets/argilla/cloud_assistant_questions/raw/main/argilla_cloud.txt
documents = loader.load_data("argilla_cloud.txt")

# setup Llama index context
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

# index the document and setup the engine
index = VectorStoreIndex.from_documents(documents, service_context=gpt_35_context)
query_engine = index.as_query_engine(similarity_top_k=2)


contexts = []
answers = []
questions = dataset["train"]["question"]

# inference over the questions
for question in tqdm(questions):
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [ ]:
# Show an example of q, a, and context
print(f"Question: {questions[0]}")
print(f"Answer: {answers[0]}")
print(f"Context: {contexts[0]}")

Question: What is the ticketing system used by Argilla for customer support?
Answer: The ticketing system used by Argilla for customer support is not specified in the given context information.
Context: ["This process ensures the client administrator has full control over their team's access and can manage their workspace efficiently.Plans The plans for the Argilla Cloud service depend on the volume of records processed, with several tiers available to suit varying needs.Each tier has a corresponding monthly and annual price, with a 10% discount applied to the annual pricing option.The tier selection and associated price will be determined by the client's selection in the Service Order Form section of the Terms of Service document.Plans are: Starter 1 Million records Base 3 Million records Medium 4 Million records Large 6 million records\n\nSupport Argilla Cloud offers comprehensive support services to address various issues that may arise during the use of our service.Support levels a

## Create Argilla dataset and collect feedback


We setup an Argilla Dataset for gathering human feedback.


For fine-tuning, we need to setup a text question to gather the human written or edited responses. This data is known as completion or demonstration data.

Additionally, leveraging the multi-aspect feedback capabilities of Argilla, we set up two additional feedback dimensions to rate the relevance of the question (as they're synthetic they might be irrelevant or bad quality) and the quality of the context retrieved from our retriever component (can be used to improve the RAG configuration).



In [ ]:
dataset = rg.FeedbackDataset(
    fields=[rg.TextField(name="user-message"), rg.TextField(name="context")],
    questions=[
        rg.RatingQuestion(name="question-rating", title="Rate the relevance of the user question", values=[1,2,3,4,5], required=False),
        rg.RatingQuestion(name="context-rating", title="Rate the quality and relevancy of context for the assistant", values=[1,2,3,4,5], required=False),
        rg.TextQuestion(name="response", title="Write a helpful, harmless, accurate response to the user question"),
    ]
)

We use the questions, context, and generated responses to build our feedback records. We pre-fill the responses in the UI with OpenAI's responses using `suggestions` and ask our labelers to edit them if neccesary.

In [ ]:
records = []

for question, answer, context in tqdm(zip(questions, answers, contexts), total=len(questions)):
  # Instantiate the FeedbackRecord
  feedback_record = rg.FeedbackRecord(
      fields={"user-message": question, "context": "\n".join(context)},
      suggestions=[
          {
              "question_name": "response",
              "value": answer,
          }
      ]
  )
  records.append(feedback_record)

# Publish dataset in Argilla UI
dataset = dataset.push_to_argilla(name="customer_assistant", workspace="admin")
dataset.add_records(records)

# optional: store and version dataset in the Hub
#dataset = dataset.push_to_huggingface("argilla/rg_customer_assistant")

Now, the dataset is available for collecting feedback with the Argilla UI. Here's a video showing the workflow for labelers:

<iframe width="100%" height="450" src="https://www.youtube.com/embed/UoMZOxFXfKI" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

## Prepare Argilla dataset for fine-tuning


We now read the responses from Argilla and prepare the dataset for fine-tuning following the [fine-tuning format from OpenAI guides](https://platform.openai.com/docs/guides/fine-tuning).

We use the a quick adaptation of LlamaIndex's `TEXT_QA_PROMPT` system prompt and the fine-tuned responses from our Argilla dataset.

In [ ]:
# If you have skipped the previous step run this to get the pre-built dataset
# dataset = rg.FeedbackDataset.from_argilla("customer_assistant", workspace="admin")

In [ ]:
# adaptation from LlamaIndex's TEXT_QA_PROMPT_TMPL_MSGS[1].content
user_message_prompt ="""Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge but keeping your Argilla Cloud assistant style, answer the query.
Query: {query_str}
Answer:
"""
# adapation from LlamaIndex's TEXT_QA_SYSTEM_PROMPT
system_prompt = """You are an expert customer service assistant for the Argilla Cloud product that is trusted around the world.
Always answer the query using the provided context information, and not prior knowledge.
Some rules to follow:
1. Never directly reference the given context in your answer.
2. Avoid statements like 'Based on the context, ...' or 'The context information ...' or anything along those lines.
"""

In [ ]:
training_examples = []

for record in dataset.records:
  responses = [response for response in record.responses if response.status == "submitted"]
  if len(responses) > 0:
    user_message = user_message_prompt.format(context_str=record.fields["context"], query_str=record.fields["user-message"])
    example = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": responses[0].values["response"].value.strip()},
    ]
    training_examples.append({"messages": example})

output_file_path = 'fine-tuning-data.jsonl'

data = training_examples

with open(output_file_path, 'w') as f:
  for sublist in data:
    json.dump(sublist, f)
    f.write('\n')

## Fine-tune GPT3.5 with high quality feedback


We fine-tune `gpt-3.5-turbo` with the exported dataset.

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

# comes from https://github.com/jerryjliu/llama_index/blob/main/experimental/openai_fine_tuning/launch_training.py
def launch_training(data_path: str) -> None:
    file_name = os.path.basename(data_path)

    # upload file
    with open(data_path, "rb") as f:
        output = openai.File.create(
            file=f,
            purpose="fine-tune",
            user_provided_filename=file_name,
        )
    print("File uploaded...")

    # launch training
    while True:
        try:
            openai.FineTuningJob.create(
                training_file=output["id"], model="gpt-3.5-turbo"
            )
            break
        except openai.error.InvalidRequestError:
            print("Waiting for file to be ready...")
            time.sleep(60)
    print(
        f"Training job {output['id']} launched. You will be emailed when it's complete."
    )

launch_training("/content/fine-tuning-data.jsonl")

## Evaluating base vs fine-tuned with human preference data

We setup a new feedback dataset for gathering human feedback to evaluate the fine-tuned model against the base model, using the test dataset.

There are many ways to collect feedback for this phase. The most suitable in this case is human preference data over responses from the two models: *asking our labelers which response is the most accurate and helpful*. We can easily do this with Argilla's `RankingQuestion`.

Additionally, as both responses can be equally bad, we can ask labelers to write down a correct response. In this case, we would be collecting demonstration data to add to our fine-tuning workflow.


### Create dataset and collect feedback

We setup and publish a new dataset with a `RankingQuestion` and `TextQuestion`, showing our labelers the `user-message` and two responses (from the base and the fine-tuned models).

In [ ]:
dataset = rg.FeedbackDataset(
    fields=[rg.TextField(name="user-message"), rg.TextField(name="response-a"), rg.TextField(name="response-b")],
    questions=[
        rg.RankingQuestion(name="preference", title="Which response is more helpful, harmless, and accurate.", values=["response-a", "response-b"]),
        rg.TextQuestion(name="response", title="If none is good, write a helpful, harmless, accurate response to the user question", required=False),
    ]
)

In [ ]:
# read our test questions
questions = load_dataset("argilla/cloud_assistant_questions", split="test")["question"]

In [ ]:
# generate responses with base model
index = VectorStoreIndex.from_documents(documents, service_context=base_context)
query_engine = index.as_query_engine(similarity_top_k=2)

contexts = []
base_model_responses = []

for question in tqdm(questions):
  response = query_engine.query(question)
  base_model_responses.append(str(response))

In [ ]:
# generate responses with ft model
index = VectorStoreIndex.from_documents(documents, service_context=ft_context)
query_engine = index.as_query_engine(similarity_top_k=2)

contexts = []
ft_model_responses = []

for question in tqdm(questions):
  response = query_engine.query(question)
  ft_model_responses.append(str(response))

An important step here is to randomize the order in which responses are shown.

If we show the fine-tuned model response always as the first option, we can introduce position bias (labelers always choosing a certain position) or making it evident to users there are two clearly different models.

To avoid this, we randomize the position and keep two metadata fields indicating which model has produced `response-1` and `response-2`. When collecting the responses, we'll use this medata to map the ranking with each model.

In [ ]:
records = []
for base, ft, question in zip(base_model_responses, ft_model_responses, questions):
  # randomizing the position is a highly important step to mitigate labeler biases
  # Shuffle the order of base and ft
  response_a, response_b = random.sample([base, ft], 2)

  # Map the responses back to their model names
  models = {
    base: "base_model",
    ft: "ft_model"
  }
  feedback_record = rg.FeedbackRecord(
      fields={"user-message": question, "response-a": response_a, "response-b": response_b},
      metadata={"response-a-model": models[response_a], "response-b-model": models[response_b]}
  )

  records.append(feedback_record)

dataset = dataset.push_to_argilla(name="finetuned-vs-base-preference", workspace="admin")
dataset.add_records(records)

Now, the dataset is available for collecting feedback with the Argilla UI. Here's a video showing the workflow for labelers:

<iframe width="100%" height="450" src="https://www.youtube.com/embed/UUK3OB0DH0Q" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

### Retrieve and analyze responses

We can dynamically collect the responses from our labelers. In this case, we will compute the win rate and ties (as users can indicate both responses are equally good or bad).

For the tutorial, we only have one user but Argilla Feedback is fully multi-user, which means you can collect feedback from several users for each data point, increasing the quality of the evaluation. 

You can read more about multi-user scenarios and built-in unification methods [on this guide](https://docs.argilla.io/en/latest/guides/llms/practical_guides/collect_responses.html).

With a very small evaluation set, we can see that the fine-tuned model responses are preferred ~60% of the time, 3x over the base model, and they are both equally good or bad ~20% of the time. 


Even with a very small fine-tuning and evaluation dataset, this already shows **promising benefits of fine-tuning models for enhancing RAG systems**.


![Fine-tuned model vs base model](../../../_static/images/llms/fine-tuning-openai-rag-feedback/fine-tune-vs-base.png "Fine-tuned model vs base model")

In [ ]:
# retrieve the dataset from Argilla
dataset = rg.FeedbackDataset.from_argilla(name="finetuned-vs-base-preference", workspace="admin")

win_rates = {
  'ft_model': 0,
  'base_model': 0,
  'tie': 0
}

# compute the win and tie rates
for record in dataset.records:
  if len(record.responses) > 0:
    for response in record.responses:
      model_a = record.metadata["response-a-model"]
      model_b = record.metadata["response-b-model"]
      preference = response.values['preference'].value
      if preference[0].rank > preference[1].rank:
        win_rates[model_a] = win_rates[model_a] + 1
      elif preference[1].rank > preference[0].rank:
        win_rates[model_b] = win_rates[model_b] + 1
      else:
        win_rates['tie'] = win_rates['tie'] + 1
win_rates
# {'ft_model': 17, 'base_model': 6, 'tie': 5}

In [ ]:
# let's make the labels more explicit
data = {'gpt3.5-fine-tuned': 17, 'gpt3.5-base': 6, 'tie': 5}
total = sum(data.values())

# Calculate percentages
percentages = [value / total * 100 for value in data.values()]

# Settings
colors = ['blue', 'grey', 'black']
labels = [f"{key} ({value:.2f}%)" for key, value in zip(data.keys(), percentages)]

# Plotting
plt.figure(figsize=(12, 2))

# The cumulative percentage is used to shift the starting point of each subsequent segment
cumulative_percentages = 0

for percent, color, label in zip(percentages, colors, labels):
    plt.barh('Models', percent, color=color, label=label, left=cumulative_percentages)
    plt.text(cumulative_percentages + percent/2, 0, label, ha='center', va='center', color='white', fontsize=10)
    cumulative_percentages += percent

plt.gca().axes.get_yaxis().set_visible(False)
plt.xlim(0, 100)
plt.title('Model Win Rates')
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.25), ncol=3)
plt.tight_layout()

# Display
plt.show()

## Appendix: Generating questions with Llama Index


We use the `DatasetGenerator` from Llama Index for generating a set of questions using a document about Argilla Cloud.

In [ ]:
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)
loader = UnstructuredReader()

# You can download this doc from: https://huggingface.co/datasets/argilla/cloud_assistant_questions/raw/main/argilla_cloud.txt
documents = loader.load_data("argilla_cloud.txt")


gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.4),
    chunk_size=60
)

In [ ]:
question_gen_query = (
    "You are customer support and sales expert of Argilla. Your task is to setup "
    "a set of frequently asked questions about the Argilla Cloud service, offer and plans"
    "formulate a single question that could be asked by a potential B2B client interested in Argilla Cloud "
    ". Restrict the question to the context information provided and don't ask general questions not related to the service and the context provided."
)

dataset_generator = DatasetGenerator.from_documents(
    documents,
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
    num_questions_per_chunk=100
)

questions = dataset_generator.generate_questions_from_nodes(num=300)